In [25]:
bits = 128

def safe_prime(nbits):
    while True:
        p = random_prime(2^nbits-1, false, 2^(nbits-1))
        if ZZ((p+1)/2).is_prime():
            return p
        
N = safe_prime(bits)
F = FiniteField(N)
g = F.multiplicative_generator()

In [26]:
x = randint(1, N - 1)
beta = g^x
r = randint(1, N - 1)
gamma = g^r
gamma

93847040391840112217458180982480181197

In [27]:
b = randint(0, 1)

In [28]:
y = r + b * x

In [29]:
g^y == gamma * beta^b

True